<a href="https://colab.research.google.com/github/Cezari0o/PLN-Trab/blob/main/a4/A4_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import math

In [5]:
def activate_fn(val, type, t):

  functions = [
               ('degrau', lambda v: 1 if v >= t else 0),
               ('sinal', lambda v: 1 if v >= 0 else -1),
               ('sigmoide', lambda v: 1 / (1 + math.exp(-v)))
               ]

  
  for n, fn in functions:
    if n == type:
      return fn(val)

  raise Exception('Nao tem a funcao')

# Dados do problema

Definindo as entradas e labels

In [2]:
X = [[0,0], [0, 1], [1, 0], [1, 1]]
labels = [0,1,1,0]

Iniciando os pesos. Cada posicao do dicionário `pesos` se refere a um neurônio, cada um com seus pesos.

In [113]:
pesos = {}
num_neuronios = 3
# Inicializando os pesos
for i in range(0, num_neuronios):
  pesos[i] = [np.random.uniform(0, 1) for i in range(2)]

pesos

{0: [0.4347768355536238, 0.5587434541315112],
 1: [0.0626647815950907, 0.6721257372388915],
 2: [0.15683363832059516, 0.4265781308632429]}

Definindo a fase de propagação.

In [116]:
# Cada camada abaixo indica quais sao os seus neuronios
camadas = {
    0: [0, 1],
    1: [2],
}

saidas = {}

net = 0
for num_amostra, amostra in enumerate(X):
  output_camadas = {0: dict(), 1: dict()}

  # Primeira camada
  for n in camadas[0]:
    net = 0
    for idx, ent in enumerate(amostra):
      net += ent * pesos[camadas[0][n]][idx]
    
    output_camadas[0][n] = activate_fn(net, 'sigmoide', 0.5)
  
  # Segunda camada
  net = 0
  for idx, s in enumerate(output_camadas[0]):
    net += output_camadas[0][s] * pesos[camadas[1][0]][idx]

  output_camadas[1][2] = activate_fn(net, 'sigmoide', 0.5)
  saidas[num_amostra] = output_camadas


for amostra in saidas:
  print(f" - Amostra {amostra} - ")
  for cam in saidas[amostra]:
    print(f" (Camada {cam})")
    for n in saidas[amostra][cam]:
      print(f"\tNeur. {n} : {saidas[amostra][cam][n]}")
  print()
# saidas

 - Amostra 0 - 
 (Camada 0)
	Neur. 0 : 0.5
	Neur. 1 : 0.5
 (Camada 1)
	Neur. 2 : 0.5011302211562537

 - Amostra 1 - 
 (Camada 0)
	Neur. 0 : 0.6361617499932771
	Neur. 1 : 0.6619789837973248
 (Camada 1)
	Neur. 2 : 0.501469192604389

 - Amostra 2 - 
 (Camada 0)
	Neur. 0 : 0.6070137538982875
	Neur. 1 : 0.5156610708120624
 (Camada 1)
	Neur. 2 : 0.5012617643847496

 - Amostra 3 - 
 (Camada 0)
	Neur. 0 : 0.7297826844629853
	Neur. 1 : 0.675855637373177
 (Camada 1)
	Neur. 2 : 0.5015844848156913



Fase de retropropagação. Infelizmente, não consegui implementar ela completamente. Faltou calcular o gradiente e o ajuste dos pesos para a camada 0 (a camada de input). Do jeito que ela se encontra abaixo, consigo apenas fazer o ajuste de pesos da camada 1 para a 2 (camada interna para a de saída).

In [115]:
lr = 0.2

print("Antes:",pesos)
for t in range(2000):
  # Calculando o erro na camada de saida
  for num_amostra, amostra in enumerate(X):
    la = labels[num_amostra]
    s  = saidas[num_amostra][1][2]

    # Obtendo o erro na camada de saida (camada 1)
    grad_layer2 = (1 - s)*s * (la - s)

    grad_layer1 = dict()

    # Obtendo o erro na camada 0
    for neuron in camadas[0]:
      s = saidas[num_amostra][0][neuron]
      grad_layer1[neuron] = s * (1 - s) * (grad_layer2 * pesos[camadas[1][0]][neuron])
      # grad_layer1[neuron] = layer1_o[neuron]*grad_layer2*layer2[neuron]

    for neuron in camadas[1]:
      for p in range(0, len(pesos[neuron])):
        pesos[neuron][p] += lr * grad_layer1[p] * saidas[num_amostra][0][p]

print("Depois:",pesos)

Antes: {0: [0.4347768355536238, 0.5587434541315112], 1: [0.0626647815950907, 0.6721257372388915], 2: [0.15683363832059516, 0.4265781308632429]}
Depois: {0: [0.4347768355536238, 0.5587434541315112], 1: [0.0626647815950907, 0.6721257372388915], 2: [0.004209749346386546, 0.004832035303629646]}
